In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import time
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
start = time.time()

PyTorch Version:  1.7.0
Torchvision Version:  0.8.1


In [2]:
import csv
import shutil

source_files = '/home/asim/Desktop/dog_breed/train/'
train_destination_folder = '/home/asim/Desktop/dog_breed/data/train/'
val_destination_folder = '/home/asim/Desktop/dog_breed/data/val/'
try:
    shutil.rmtree(train_destination_folder)
    shutil.rmtree(val_destination_folder)
except:
    print("weew")
os.mkdir(train_destination_folder)
os.mkdir(val_destination_folder)
with open("labels.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    index = 0
    for i, line in enumerate(reader):
        
        temp = line[0].split(',')
        name = temp[0]
        breed = temp[1]
#         print(name,breed)
        
        if name != "id":
            directory  = os.path.isdir(train_destination_folder + breed)
            if directory ==True:
                no_of_files = len(os.listdir(train_destination_folder+breed)) 
#                 print(no_of_files)
              
                if index%5 != 0:
                    
                    shutil.copy(source_files + name + ".jpg", train_destination_folder + breed )
                    index +=1
                else:
                    
                    val_directory  = os.path.isdir(val_destination_folder + breed)
                    if val_directory == True:
                        shutil.copy(source_files + name + ".jpg", val_destination_folder + breed )
                        index += 1
                    else:
                        os.mkdir(val_destination_folder + breed)
                        shutil.copy(source_files + name + ".jpg", val_destination_folder + breed)
                        index += 1
            else:
                os.mkdir(train_destination_folder + breed)
                shutil.copy(source_files + name + ".jpg", train_destination_folder + breed)
                index +=1 
              
       

        
        

In [3]:
data_dir = "./data/"


model_name = "resnet"

# Number of classes in the dataset
num_classes = len(os.listdir(train_destination_folder))

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True


In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


In [5]:
def set_parameter_requires_grad(model, feature_extracting):
        if feature_extracting:
            for param in model.parameters():
                param.requires_grad = False

In [6]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    return model_ft, input_size

In [7]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)


# %%
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cpu")


# %%
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


# %%
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))


# %%
torch.save(model_ft, 'inception_floorplan.pth')
end = time.time()
print(end - start)
  

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

train Loss: 3.5823 Acc: 0.2623
val Loss: 1.5340 Acc: 0.6677

Epoch 1/49
----------
train Loss: 2.1873 Acc: 0.5402
val Loss: 0.9109 Acc: 0.7791

Epoch 2/49
----------
train Loss: 1.7666 Acc: 0.6044
val Loss: 0.7040 Acc: 0.8103

Epoch 3/49
----------
train Loss: 1.5694 Acc: 0.6266
val Loss: 0.6144 Acc: 0.8212

Epoch 4/49
----------
train Loss: 1.4526 Acc: 0.6421
val Loss: 0.5514 Acc: 0.8291

Epoch 5/49
----------
train Loss: 1.3925 Acc: 0.6555
val Loss: 0.5458 Acc: 0.8266

Epoch 6/49
----------
train Loss: 1.3405 Acc: 0.6604
val Loss: 0.5188 Acc: 0.8326

Epoch 7/49
----------
train Loss: 1.2802 Acc: 0.6705
val Loss: 0.4890 Acc: 0.8450

Epoch 8/49
----------
train Loss: 1.2763 Acc: 0.6676
val Loss: 0.4833 Acc: 0.8375

Epoch 9/49
----------
train Loss: 1.2470 Acc: 0.6762
val Loss: 0.4941 Acc: 0.8455

Epoch 10/49
----------
train Loss: 1.2197 Acc: 0.6782
val Loss: 0.4856 Acc: 0.8479

Epoch 11/49
----------
train Loss: 1.2089 Acc: 0.6811
val Loss: 0.4864 Acc: 0.8465

Epoch 12/49
----------
t

SyntaxError: 'return' outside function (<ipython-input-7-eef953771c56>, line 75)